In [14]:
# 매크로(.xlsm) 엑셀 파일 자동 열기/수정/저장/계산용 라이브러리 설치
%pip install xlwings pandas numpy openpyxl

Note: you may need to restart the kernel to use updated packages.


In [15]:
# 라이브러리 불러오기
# 엑셀파일 열기 및 수정.저장.계산 가능
import xlwings as xw
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [16]:
# 엑셀파일 연결
try:
    book = xw.Book(r"C:\Users\naya9\OneDrive\바탕 화면\포트폴리오\포트폴리오(csm).xlsm")
except Exception as e:
    # 엑셀을 이미 열어둔 경우 활성 통합문서로 붙기
    book = xw.books.active

# 매크로 호출 추가 
try:
    # 보험기간 계산 먼저 반영
    book.macro('policyterm')()
except Exception as e:
    print("매크로 실행 실패", e)
    
# 각 시트에 대한 이름만들기
inp = book.sheets['입력']
da = book.sheets['데이터']
cal = book.sheets['계산']
out = book.sheets['결과출력']

# 매크로가 숫자형으로 읽은 보험기간 값 확인
policy_term = inp['C5'].value
print(f"보험기간(년): {policy_term}")



보험기간(년): 5.0


In [17]:
# 엑셀파일 연결
try:
    book = xw.Book(r"C:\Users\naya9\OneDrive\바탕 화면\포트폴리오\포트폴리오(csm).xlsm")
except Exception as e:
    # 엑셀을 이미 열어둔 경우 활성 통합문서로 붙기
    book = xw.books.active
    
# 각 시트에 대한 이름만들기
inp = book.sheets['입력']
da = book.sheets['데이터']
cal = book.sheets['계산']
out = book.sheets['결과출력']

In [18]:
# xlwings 유틸 함수: get_value_by_label / refresh_once / start_polling
import time
from typing import Optional, Dict
# 항목명으로 값(3번째 열)을 찾는 유틸리티
# 입력 시트에서 레이블(B열)로 값을(C열) 찾기
def get_input_value(sheet, label: str, label_col: str = "B", value_col: str = "C", max_rows: int = 200):
    rng_labels = sheet[f"{label_col}1:{label_col}{max_rows}"]
    for cell in rng_labels:
        lv = str(cell.value).strip().replace(" ", "").replace("\n", "") if cell.value is not None else ""
        if lv == label.replace(" ", "").replace("\n", ""):
            return sheet[f"{value_col}{cell.row}"].value
    return None

LABELS = {
    "policyholder_age": "피보험자 나이(세)",
    "policy_term": "보험기간(년)",
    "annual_premium": "연납보험료",
    "death_benefit": "사망보험금",
    "discount_rate": "할인율",
    "lapse_rate": "해약률",
    "expense_ratio": "사업비율",
    "direct_expense": "직접사업비",
    "surrender_value": "해약환급금",
    "mortality_rate": "사망률",
}
# 입력시트에 있는 레이블을 읽어 계산하고 결과출력 시트에 기록
# 이미 연결된 inp/out을 사용해 한 번 읽고 계산/기록
def refresh_once() -> Dict[str, Optional[float]]:
    values: Dict[str, Optional[float]] = {}
    for key, label in LABELS.items():
        values[key] = get_input_value(inp, label)

    ap = values.get("annual_premium") or 0
    dr = values.get("discount_rate") or 0
    calc_result = ap * (1 + dr)
    out["B2"].value = calc_result
    return values

# 주기적 폴링
# 1초마다 입력시트에 있는 레이블을 읽어 계산하고 결과출력 시트에 기록
def start_polling(interval_sec: float = 1.0, iterations: Optional[int] = None):
    n = 0
    try:
        while True:
            vals = refresh_once()
            print({k: vals.get(k) for k in ("annual_premium", "discount_rate", "mortality_rate")})
            time.sleep(interval_sec)
            n += 1
            if iterations is not None and n >= iterations:
                break
    except KeyboardInterrupt:
        print("폴링 중단")

print("입력시트 레이블 읽기 준비 완료")

입력시트 레이블 읽기 준비 완료


In [64]:
# 보험기간 값이 있는 '입력' 시트의 주소 찾기
try:
    addr
except NameError:
    def find_input_value_address(sheet, label: str, label_col: str = "B", value_col: str = "C", max_rows: int = 200):
        rng_labels = sheet[f"{label_col}1:{label_col}{max_rows}"]
        for cell in rng_labels:
            lv = str(cell.value).strip().replace(" ", "").replace("\n", "") if cell.value is not None else ""
            if lv == label.replace(" ", "").replace("\n", ""):
                return f"{value_col}{cell.row}"
        return None
addr = find_input_value_address(inp, "보험기간(년)")

if addr is None:
    raise ValueError("입력 시트에서 '보험기간' 주소를 찾지 못했습니다.")

# B1 기준, 오른쪽으로 보험기간 수만큼 수식 채우기
max_cols = 400
formula = f'=IF(COLUMN()-1<=\'{inp.name}\'!{addr}, COLUMN()-1 & "차년도", "")'
rg = out.range("B1").resize(1, max_cols)
# 같은 수식을 전체에 채우면 COLUMN()이 각 셀 기준으로 평가되어 1,2,3...차년도 형태로 동작
rg.formula = [[formula] * max_cols]

# A1부터 같은 열에 요청 라벨 기입
labels = [
    "구분",
    "PV(유입)_보험료",
    "PV(유출)_보험금",
    "최선추정부채(BEL)",
    "위험조정(RA)",
    "이행현금흐름(FCF)",
    "보험계약마진(CSM)",
    "보험계약부채",
]
out.range("A1").value = [[lbl] for lbl in labels]


In [80]:
# 계산작업을 위해 입력 시트 레이블 값 일괄 가져오기 및 표시
# 입력 시트 레이블 값 일괄 가져오기 및 표시
label_to_value = {}
for key, label in LABELS.items():
    label_to_value[label] = get_input_value(inp, label)

# 편의용 변수 바인딩
issue_age = label_to_value.get("피보험자 나이(세)")
policy_term = label_to_value.get("보험기간(년)")
annual_premium = label_to_value.get("연납보험료")
death_benefit = label_to_value.get("사망보험금")
discount_rate = label_to_value.get("할인율")
lapse_rate = label_to_value.get("해약률")
expense_ratio = label_to_value.get("사업비율")
direct_expense = label_to_value.get("직접사업비")
surrender_value = label_to_value.get("해약환급금")
mortality_rate = label_to_value.get("사망률")

print(label_to_value)

{'피보험자 나이(세)': 24.0, '보험기간(년)': 7.0, '연납보험료': 1200.0, '사망보험금': 10000.0, '할인율': 0.030312358683219433, '해약률': 0.02059992899344719, '사업비율': 0.1223, '직접사업비': 0.00010191666666666667, '해약환급금': 1335269.7727272727, '사망률': 0.00032}


In [ ]:
# 사망률 데이터시트 
# da = book.sheets['데이터']
data_frame = da.range("A2").expand().value
df = pd.DataFrame(data_frame[1:], columns = data_frame[0])
# 필요한 정보만 추출하기 위해 앞에 index 제거
print(df.to_string(index = False))

In [85]:
from pathlib import Path
# 입력시트에서 피보험자 나이를 입력하면 데이터시트에서 해당 나이의 사망률 찾기
def resolve_row_and_value(issue_age):
    book = xw.books.active
    file_path = Path(book.fullname)
    inp = book.sheets['입력']

    gender = str(inp['C3'].value or "").strip()  # "남"/"여"

    df = pd.read_excel(
        file_path, sheet_name="데이터", header=None,
        usecols="A:C", skiprows=2, engine="openpyxl"
    )

    # 데이터시트에서 A열에 있는 나이를 숫자로 통일해 매칭
    a_num = pd.to_numeric(
        df[0].astype(str).str.replace(r"[^0-9.]", "", regex=True),
        errors="coerce"
    ).astype("Int64")
    mask = a_num.eq(issue_age)
    if not mask.any():
        return None, None, None

# 매칭되는 행과 열 찾기
    row_idx = mask[mask].index[0]      # pandas 인덱스(A3 기준 0)
    excel_row = row_idx + 3            # 실제 엑셀 행 번호
    col_letter = "C" if gender == "남" else "B"
    col_idx = 2 if gender == "남" else 1  # pandas col idx (B/C)

    value = df.iat[row_idx, col_idx]   # 해당 셀 값
    return excel_row, col_letter, value

# 매번 최신 값으로 갱신
excel_row, col_letter, value = resolve_row_and_value(issue_age)

# 입력!C3가 "남"이면 da시트 B열, "여"이면 C열 사용
wb = xw.books.active
inp = wb.sheets["입력"]
da  = wb.sheets["데이터"]

gender = str(inp["C3"].value).strip()          # "남" / "여"
col_letter = "C" if gender == "남" else "B"    # 남→B열, 여→C열

wb = xw.books.active
da = wb.sheets["데이터"]         

# 매칭된 행과 열을 종합하여 사망률값 찾기
value = da[f"{col_letter}{excel_row}"].value
print("사망률:", value)


사망률: 0.00032


In [ ]:
# 사망률을 반영한 사망보험금을 계산하기 위해 보험기간 만큼 아래로 사망률 값들 가져오기
inp = wb.sheets['입력']
term = int(float(label_to_value.get("보험기간(년)") or inp['C5'].value))  # 기간 숫자

rng = da[f"{col_letter}{excel_row}"].resize(term, 1)
vals = rng.value  # xlwings: (term x 1) 범위 값

# 사망률표에서 필요한 열만 추출하여 리스트로 정규화하기
if isinstance(vals, list) and term > 1 and isinstance(vals[0], list):
    mortality_series = [v[0] for v in vals]
else:
    mortality_series = [vals] if term == 1 else vals

print("보험기간을 반영한 사망률 목록:", mortality_series)

보험기간을 반영한 사망률 목록: [0.00032, 0.00033, 0.00033, 0.00032, 0.00033, 0.00036, 0.00039]
